In [76]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D , MaxPooling2D

import os

In [77]:
train_dir = "/kaggle/input/fer2013/train"
test_dir = "/kaggle/input/fer2013/test"
os.listdir(train_dir)

['surprise', 'fear', 'angry', 'neutral', 'sad', 'disgust', 'happy']

In [78]:
train_datagen= ImageDataGenerator(
    rescale =1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest"
)
val_datagen=ImageDataGenerator(
    rescale =1./255,
   
)
train_data = train_datagen.flow_from_directory(
    train_dir,
    color_mode="grayscale",
    target_size=(48,48),
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
)
val_data = val_datagen.flow_from_directory(
    test_dir,
    color_mode="grayscale",
    target_size=(48,48),
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [79]:
img,label=train_data.__next__()

In [80]:

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation="relu",input_shape=(train_data.image_shape)))
model.add(Conv2D(64,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))
model.add(Conv2D(128,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))
model.add(Conv2D(256,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(train_data.num_classes,activation="softmax"))

In [81]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_58 (Conv2D)              │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 44, 44, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_60 (Conv2D)              │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_37 (MaxPooling2D) │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_61 (Conv2D)              │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_38 (MaxPooling2D) │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,489,095 (9.50 MB)

 Trainable params: 2,489,095 (9.50 MB)

 Non-trainable params: 0 (0.00 B)

In [83]:
total_train_images=0
for root,dirs,files in os.walk(train_dir):
   total_train_images += len(files)
total_test_images=0
for root,dirs,files in os.walk(test_dir):
   total_test_images += len(files)
total_train_images,total_test_images    
    

(28709, 7178)

In [84]:
history=model.fit(train_data,
                  steps_per_epoch=total_train_images//32,
                  epochs=30,
validation_data = val_data,
validation_steps=total_test_images//32,
                 )

Epoch 1/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 82s 86ms/step - accuracy: 0.2541 - loss: 1.8134 - val_accuracy: 0.3306 - val_loss: 1.6782
Epoch 2/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.2188 - loss: 1.8564 - val_accuracy: 0.3224 - val_loss: 1.6883
Epoch 3/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 46s 51ms/step - accuracy: 0.3151 - loss: 1.7060 - val_accuracy: 0.3972 - val_loss: 1.5829
Epoch 4/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.3125 - loss: 1.5187 - val_accuracy: 0.3958 - val_loss: 1.5845
Epoch 5/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 45s 50ms/step - accuracy: 0.3641 - loss: 1.6200 - val_accuracy: 0.4353 - val_loss: 1.4725
Epoch 6/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.2812 - loss: 1.8964 - val_accuracy: 0.4360 - val_loss: 1.4739
Epoch 7/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 46s 51ms/step - accuracy: 0.3978 - loss: 1.5558 - val_accuracy: 0.4615 - val_loss: 1.4144
Epoch 8/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.3750 - loss: 1.5259 - val_acc

In [86]:
history=model.fit(train_data,
                  steps_per_epoch=total_train_images//32,
                  epochs=60,
                  initial_epoch=30,
validation_data = val_data,
validation_steps=total_test_images//32,
                 )

Epoch 31/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 46s 51ms/step - accuracy: 0.5443 - loss: 1.2005 - val_accuracy: 0.5738 - val_loss: 1.1178
Epoch 32/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.4375 - loss: 1.2915 - val_accuracy: 0.5731 - val_loss: 1.1179
Epoch 33/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 44s 49ms/step - accuracy: 0.5525 - loss: 1.1804 - val_accuracy: 0.5699 - val_loss: 1.1259
Epoch 34/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.4062 - loss: 1.5322 - val_accuracy: 0.5682 - val_loss: 1.1297
Epoch 35/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 46s 51ms/step - accuracy: 0.5568 - loss: 1.1700 - val_accuracy: 0.5780 - val_loss: 1.1034
Epoch 36/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5000 - loss: 1.1580 - val_accuracy: 0.5770 - val_loss: 1.1029
Epoch 37/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 47s 52ms/step - accuracy: 0.5629 - loss: 1.1592 - val_accuracy: 0.5818 - val_loss: 1.0967
Epoch 38/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.4375 - loss: 1.4174 - 

In [87]:
history=model.fit(train_data,
                  steps_per_epoch=total_train_images//32,
                  epochs=90,
                  initial_epoch=60,
validation_data = val_data,
validation_steps=total_test_images//32,
                 )

Epoch 61/90
897/897 ━━━━━━━━━━━━━━━━━━━━ 56s 62ms/step - accuracy: 0.5974 - loss: 1.0585 - val_accuracy: 0.6123 - val_loss: 1.0366
Epoch 62/90
897/897 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.6562 - loss: 0.9884 - val_accuracy: 0.6134 - val_loss: 1.0386
Epoch 63/90
897/897 ━━━━━━━━━━━━━━━━━━━━ 52s 58ms/step - accuracy: 0.6050 - loss: 1.0495 - val_accuracy: 0.6051 - val_loss: 1.0532
Epoch 64/90
897/897 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6250 - loss: 0.9320 - val_accuracy: 0.6051 - val_loss: 1.0550
Epoch 65/90
897/897 ━━━━━━━━━━━━━━━━━━━━ 47s 52ms/step - accuracy: 0.6043 - loss: 1.0443 - val_accuracy: 0.6173 - val_loss: 1.0208
Epoch 66/90
897/897 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.6250 - loss: 0.9604 - val_accuracy: 0.6170 - val_loss: 1.0200
Epoch 67/90
897/897 ━━━━━━━━━━━━━━━━━━━━ 41s 46ms/step - accuracy: 0.6091 - loss: 1.0482 - val_accuracy: 0.6138 - val_loss: 1.0360
Epoch 68/90
897/897 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.6562 - loss: 0.9402 -

In [88]:
history=model.fit(train_data,
                  steps_per_epoch=total_train_images//32,
                  epochs=110,
                  initial_epoch=90,
validation_data = val_data,
validation_steps=total_test_images//32,
                 )

Epoch 91/110
897/897 ━━━━━━━━━━━━━━━━━━━━ 44s 49ms/step - accuracy: 0.6333 - loss: 0.9787 - val_accuracy: 0.6331 - val_loss: 1.0096
Epoch 92/110
897/897 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.5000 - loss: 1.1951 - val_accuracy: 0.6320 - val_loss: 1.0097
Epoch 93/110
897/897 ━━━━━━━━━━━━━━━━━━━━ 42s 47ms/step - accuracy: 0.6393 - loss: 0.9654 - val_accuracy: 0.6327 - val_loss: 1.0103
Epoch 94/110
897/897 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7188 - loss: 0.7690 - val_accuracy: 0.6321 - val_loss: 1.0119
Epoch 95/110
897/897 ━━━━━━━━━━━━━━━━━━━━ 41s 46ms/step - accuracy: 0.6370 - loss: 0.9615 - val_accuracy: 0.6338 - val_loss: 1.0110
Epoch 96/110
897/897 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.4688 - loss: 1.1063 - val_accuracy: 0.6342 - val_loss: 1.0108
Epoch 97/110
897/897 ━━━━━━━━━━━━━━━━━━━━ 44s 49ms/step - accuracy: 0.6431 - loss: 0.9559 - val_accuracy: 0.6335 - val_loss: 1.0035
Epoch 98/110
897/897 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.5625 - loss: 1

In [90]:
model.save("emotion_recognition_CNN.h5")